In [112]:
import pandas as pd
import tensorflow_text as text
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import os
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences

from collections import Counter
from keras.models import Sequential 
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

from keras import optimizers
from keras.backend import binary_crossentropy
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional

In [60]:
df= pd.read_csv("tweet_nlp_data_cleaned.csv")
df

,text,label,clean_tweet,text_normalization,word_tokenize,lemmatize_words
0,@BookstAnna Uncover the truth.\nStream #HOTD t...,HOTD,uncover the truth stream tonight at 9pm on rep...,uncover truth stream tonight 9pm reply,"['reply', 'truth', 'stream', 'tonight', 'uncov...","['reply', 'truth', 'stream', 'tonight', 'uncov..."
1,RT @arishatistic: the kiss we deserved\n\n#hot...,HOTD,the kiss we deserved,kiss deserved,"['kiss', 'deserved']","['kiss', 'deserved']"
2,RT @caffrey05: Ya quiero que se muera Viserys ...,HOTD,I already want Viserys to die so that the real...,I already want Viserys die real chaos begin ho...,"['real', 'Viserys', 'begin', 'die', 'I', 'chao...","['real', 'Viserys', 'begin', 'die', 'I', 'chao..."
3,RT @HOTDsource: More of Milly Alcock behind th...,HOTD,more of milly alcock behind the scenes of epis...,milly alcock behind scenes episode 5 adorable,"['episode', 'adorable', 'alcock', 'milly', 'sc...","['episode', 'adorable', 'alcock', 'milly', 'sc..."
4,@mfredericks1x Duty above all. \nStream #HOTD ...,HOTD,duty above all stream tonight at 9pm on reply ...,duty stream tonight 9pm reply,"['duty', 'tonight', 'stream', 'reply']","['duty', 'tonight', 'stream', 'reply']"
...,...,...,...,...,...,...
53849,Amazon’s Delay for ‘The Rings of Power’ Review...,ROP,amazon's delay for 'the rings of power' review...,amazon's delay 'the rings power' reviews prime...,"['pa', 'initiative', 'exclusive', 'filter', 't...","['pa', 'initiative', 'exclusive', 'filter', 't..."
53850,The latest #TheRingsOfPower trailer has us the...,ROP,the latest trailer has us theorizing to the mo...,latest trailer us theorizing moon back! read t...,"['back', 'thoughts', 'latest', 'read', 'us', '...","['back', 'thought', 'latest', 'read', 'u', 'th..."
53851,The Lord of the Rings: The Rings of Power Revi...,ROP,the lord of the rings the rings of power revie...,lord rings rings power review fantastic world ...,"['lord', 'world', 'search', 'power', 'rings', ...","['lord', 'world', 'search', 'power', 'ring', '..."
53852,Half way through the first episode of Rings of...,ROP,half way through the first episode of rings of...,half way first episode rings power already lov...,"['loving', 'aaaahhh', 'already', 'half', 'powe...","['loving', 'aaaahhh', 'already', 'half', 'powe..."


In [61]:
df_hotd= df[df["label"]=='HOTD']
df_hotd.shape

(26927, 6)

In [62]:
df_rop= df[df["label"]=='ROP']
df_rop.shape

(26927, 6)

In [63]:
df = df.sample(frac=1, random_state=1).reset_index()
df.drop('index', inplace=True, axis=1)
df

,text,label,clean_tweet,text_normalization,word_tokenize,lemmatize_words
0,Amazon Studios has launched “The Lord of the R...,ROP,"amazon studios has launched ""the lord of the r...","amazon studios launched ""the lord rings rings ...","['hobbit', 'take', 'tolkien', 'j', 'place', 's...","['hobbit', 'take', 'tolkien', 'j', 'place', 's..."
1,Lenny Henry praises new Lord of the Rings seri...,ROP,lenny henry praises new lord of the rings seri...,lenny henry praises new lord rings series '21s...,"['century', 'update', 'praises', 'lenny', 'lor...","['century', 'update', 'praise', 'lenny', 'lord..."
2,RT @HOTDsource: The #HouseOfTheDragon Season 1...,HOTD,the season 1 dragons so far,season 1 dragons far,"['dragons', 'far', 'season']","['dragon', 'far', 'season']"
3,RT @FreeFolkMemes: #SerCriston out here commit...,HOTD,out here committing hate crimes,committing hate crimes,"['crimes', 'committing', 'hate']","['crime', 'committing', 'hate']"
4,RT @HBOMaxLA: Un regalo para ti: Fondos de pan...,HOTD,a gift for you on wallpapers which one are you...,gift wallpapers one going use?,"['wallpapers', 'use', 'gift', 'going', 'one']","['wallpaper', 'use', 'gift', 'going', 'one']"
...,...,...,...,...,...,...
53849,@DanFriedman81 Rings of Power is getting revie...,ROP,rings of power is getting review bombed at the...,rings power getting review bombed moment fanta...,"['getting', 'pleasantly', 'far', 'moment', 'sh...","['getting', 'pleasantly', 'far', 'moment', 'sh..."
53850,"Surprise surprise, I really like the rings of ...",ROP,surprise surprise i really like the rings of p...,surprise surprise really like rings power far,"['like', 'surprise', 'really', 'power', 'rings...","['like', 'surprise', 'really', 'power', 'ring'..."
53851,RT @oochotd: The world was theirs #HouseOfTheD...,HOTD,the world was theirs,world,['world'],['world']
53852,Is House Of The Dragon based on source materia...,HOTD,is house of the dragon based on source materia...,house dragon based source material another one...,"['dragon', 'showrunners', 'stuff', 'based', 'u...","['dragon', 'showrunners', 'stuff', 'based', 'u..."


In [52]:
def counter_word(text):
  count= Counter()
  for i in text.values:
    for word in i.split():
      count[word]+=1
  return count

In [64]:
df=df.astype(str)
text= df.text_normalization
counter= counter_word(text)

In [91]:
num_words= len(counter)
df.rename(columns = {'label':'Category', 'text_normalization':'Message'}, inplace = True)
df['HOTD']= df['Category'].apply(lambda x: 1 if x== 'HOTD' else 0)
y= df.HOTD
#Max number of words in a sequence
max_lenght= 50 #We are going to have the same sequence lenght for each 

In [92]:
X_train, X_test, y_train, y_test= train_test_split(df['Message'], df['HOTD'], stratify=df['HOTD'])

In [93]:
tokenizer= Tokenizer(num_words= num_words)
tokenizer.fit_on_texts(X_train)

In [94]:
word_index= tokenizer.word_index

In [95]:
X_sequences= tokenizer.texts_to_sequences(X_train)

In [96]:
train_padded= pad_sequences(X_sequences, 
                           maxlen=max_lenght, 
                           padding='post', 
                           truncating='post')

In [97]:
test_sequences= tokenizer.texts_to_sequences(X_test)
test_padded= pad_sequences(test_sequences, 
                           maxlen=max_lenght, 
                           padding='post', 
                           truncating='post')

In [98]:
print(df.Message[7])
print(X_sequences[7])

stream rings power amazon prime
[135, 1182, 23, 1316, 17, 1067, 483, 519, 1481, 1140]


In [99]:
model= Sequential()

model.add(Embedding(num_words, 16, input_length=max_lenght))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.1, return_sequences=False)) ##32, 250
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))

optimizer= Adam()

In [100]:
sequence_length=12
BASENAME="LSTM"

In [101]:
#model_weights_path = f"results/{BASENAME}-{sequence_length}.h5"

In [102]:
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [103]:
if not os.path.isdir("results"):
    os.mkdir("results")

In [104]:
lb= LabelEncoder()
y= lb.fit_transform(y_train)
y_2= lb.fit_transform(y_test)

In [105]:
#early_stop = EarlyStopping(monitor='val_loss', patience=2)

In [106]:
model.fit(train_padded, y_train, epochs=15, validation_data=(test_padded, y_test),)

Epoch 1/15
1263/1263 [==============================] - 99s 77ms/step - loss: 0.4850 - accuracy: 0.7109 - val_loss: 0.6964 - val_accuracy: 0.5000
Epoch 2/15
1263/1263 [==============================] - 97s 77ms/step - loss: 0.4365 - accuracy: 0.7087 - val_loss: 0.0646 - val_accuracy: 0.9808
Epoch 3/15
1263/1263 [==============================] - 97s 77ms/step - loss: 0.0552 - accuracy: 0.9845 - val_loss: 0.0501 - val_accuracy: 0.9863
Epoch 4/15
1263/1263 [==============================] - 97s 77ms/step - loss: 0.0350 - accuracy: 0.9904 - val_loss: 0.0586 - val_accuracy: 0.9845
Epoch 5/15
1263/1263 [==============================] - 102s 81ms/step - loss: 0.0289 - accuracy: 0.9917 - val_loss: 0.0614 - val_accuracy: 0.9828
Epoch 6/15
1263/1263 [==============================] - 97s 77ms/step - loss: 0.0185 - accuracy: 0.9949 - val_loss: 0.0653 - val_accuracy: 0.9830
Epoch 7/15
1263/1263 [==============================] - 97s 77ms/step - loss: 0.0164 - accuracy: 0.9949 - val_loss: 0.0833 

In [107]:
score = model.evaluate(test_padded, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

421/421 [==============================] - 8s 18ms/step - loss: 0.0950 - accuracy: 0.9788
Test loss: 0.09503506869077682
Test accuracy: 0.9787581562995911


In [108]:
y_pred = model.predict(test_padded)
#y_pred=(y_pred>0.5)
print(y_pred)

421/421 [==============================] - 9s 20ms/step
[[1.3076406e-04]
 [2.2839496e-04]
 [9.9993777e-01]
 ...
 [1.5531672e-04]
 [1.7581733e-04]
 [9.9993885e-01]]


In [109]:
y_pred=(y_pred>0.5)

In [110]:
y_pred[14]

array([ True])

In [ ]:
test_padded

array([[  55,  134,  332, ...,    0,    0,    0],
       [  20,  415,  487, ...,    0,    0,    0],
       [  51, 2339, 1083, ...,    0,    0,    0],
       ...,
       [  81, 1065,  542, ...,    0,    0,    0],
       [  65,   82,   69, ...,    0,    0,    0],
       [1859,  303,    6, ...,    0,    0,    0]], dtype=int32)

In [ ]:
reverse_word_index= dict([(value,key) for (key,value) in word_index.items()])
def decode (text):
  return " ".join([reverse_word_index.get(i,"?") for i in text])

In [ ]:
decode(test_padded[14])

'black queen ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?'

In [113]:
reconstructed_model = keras.models.load_model("my_model_LSTM_classifier")

In [116]:
#score = model.evaluate(test_padded, y_test)
reconstructed_model.save("results/my_model_LSTM_classifier.h5")

In [ ]:
z_pred= reconstructed_model.predict(test_padded)
z_pred

421/421 [==============================] - 11s 25ms/step


array([[0.9998498 ],
       [0.9993107 ],
       [0.99985856],
       ...,
       [0.99989045],
       [0.9998474 ],
       [0.00472414]], dtype=float32)

In [ ]:
z_pred=(z_pred>0.5)
z_pred[14]

array([ True])

In [ ]:
decode(test_padded[14])

'black queen ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?'

In [ ]:
def word_count(text):
  count= Counter()
  for i in text:
    for word in i.split():
      count[word]+=1
  return count
  
def counter_word(text):
    counter= word_count(text)
    max_lenght= 80
    num_words= len(counter)
    #count= 80

    tokenizer= Tokenizer(num_words= num_words)
    tokenizer.fit_on_texts(text)
    word_index= tokenizer.word_index
    text_sequence = tokenizer.texts_to_sequences(text)

    
    text_padded= pad_sequences(text_sequence, 
                           maxlen=80, 
                           padding='post', 
                           truncating='post')
    return text_padded

In [ ]:
text = ['The Red Keep is under siege Seven Kingdoms he fate of the Seven Kingdoms hangs in the balance','Will House Targaryen survive','he fate of the Seven Kingdoms hangs in the balance']
counter= counter_word(text)
counter

array([[ 1, 10, 11, 12, 13, 14,  2,  3,  4,  5,  6,  1,  2,  3,  7,  8,
         1,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [15, 16, 17, 18,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 4,  5,  6,  1,  2,  3,  7,  8,  1,  9,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0

In [ ]:
y_pred = model.predict(counter)
y_pred=(y_pred>0.5)
y_pred

1/1 [==============================] - 0s 37ms/step


array([[False],
       [ True],
       [False]])

In [ ]:
decode(train_padded[0])

In [ ]:
z_pred[14]

In [ ]:
y_pred=(y_pred>0.5)

In [ ]:
y_pred

array([[ True],
       [ True],
       [False],
       ...,
       [ True],
       [ True],
       [False]])

In [ ]:
type(list_w[0][0])

float

In [ ]:
test_padded

array([[  55,  134,  332, ...,    0,    0,    0],
       [  20,  415,  487, ...,    0,    0,    0],
       [  51, 2339, 1083, ...,    0,    0,    0],
       ...,
       [  81, 1065,  542, ...,    0,    0,    0],
       [  65,   82,   69, ...,    0,    0,    0],
       [1859,  303,    6, ...,    0,    0,    0]], dtype=int32)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d578b49a-f433-4f75-ac7b-2845145b8de8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>